<a href="https://colab.research.google.com/github/Praneeth-18/NLP-POS-Substitutions/blob/main/Part_1_NLP_POS_Substitutions_and_Tones_for_Poets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
poet1 = "Rudyard Kipling"
poet2 = "T S Eliot"
store_loc = "/content/drive/MyDrive/ML-Spring-2024/team-Pi/Praneeth-poets/"
output_loc =  "/content/drive/MyDrive/ML-Spring-2024/deep-poets/Team-Pi/"

In [ ]:
!pip3 install nltk
!pip3 install sentence-transformers
!pip3 install contractions
!pip3 install pandas
!pip install bertopic
!pip install gensim
!pip install pyLDAvis
!pip install spacy
!pip install requests
!pip install beautifulsoup4
!pip install lxml
# !pip install --upgrade ipython==7.34.0 ipykernel==5.5.6

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.3/163.3 kB 4.9 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [ ]:
import requests
import pandas as pd
import bs4 as bs
import urllib.request
import json
import contractions
import re
import nltk
from nltk.tokenize import word_tokenize
from sentence_transformers import SentenceTransformer, util
from nltk.corpus import stopwords
from transformers import pipeline
from spacy import displacy
from bertopic import BERTopic
import os
import pyLDAvis
import gensim
import spacy
import gensim.corpora as corpora
from gensim.models import CoherenceModel
import pyLDAvis.gensim_models as gensimvis
from gensim.utils import simple_preprocess

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Mounted at /content/drive


In [ ]:
import warnings
warnings.filterwarnings('ignore', 'Calling')
warnings.filterwarnings('ignore', 'For a faster implementation')
warnings.filterwarnings('ignore', 'In')
warnings.filterwarnings('ignore', 'SelectableGroups dict interface')
warnings.filterwarnings('ignore', '`np')
warnings.filterwarnings('ignore', 'SelectableGroups')
warnings.filterwarnings('ignore', 'The')
# warnings.filterwarnings("ignore", category=DeprecationWarning)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('universal_tagset')
nltk.download('wordnet')
nltk.download('punkt')

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Unzipping taggers/universal_tagset.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

## Scrapping Poems for 2 Authors

In [ ]:
def clean_txt(text):
    text=text.replace(u'\xa0', u' ').replace(r'\r',u'\n').replace(r'\n', ' ').replace('\n', ' ').replace('\t', ' ')
    text = contractions.fix(text)
    text = text.strip().lower()
    text = re.sub(r"'", ' ', text)
    filters='!"\'#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n'
    translate_dict = dict((i, " ") for i in filters)
    translate_map = str.maketrans(translate_dict)
    text = text.translate(translate_map)
    text = ' '.join([w for w in text.split() if len(w)>1])
    text = re.sub(' +', ' ', text)
    text = ''.join(text)
    return text

def scrapePoemFromURL(poem_url_list):
    poems_results=[]
    for url in poem_url_list:
        header = {'User-Agent':'Mozilla/5.0'}
        requests = urllib.request.Request(url,headers=header)
        requestread = urllib.request.urlopen(requests).read()
        soup = bs.BeautifulSoup(requestread,'html.parser')
        poem_for_human = soup.find_all('div', class_="poem_body")[0].text.replace("© by owner. provided at no charge for educational purposes", "")
        poem_for_machine = (clean_txt(poem_for_human))
        poem_title = soup.find_all('h1')[0].text
        poems_results.append([poem_title, poem_for_machine, poem_for_human])
    return poems_results

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
def get_poem_url_json(poet):
    site = "https://allpoetry.com"
    search = "https://allpoetry.com/items/read_by/"
    page = 1
    URL_list = list()
    while len(URL_list) < 10:
        url = search + poet + "?page=" + str(page)
        response = requests.get(url)
        soup = bs.BeautifulSoup(response.text, 'lxml')
        poem_list = soup.find_all('a', href=True, class_='nocolor fn')
        for i in range(len(poem_list)):
            #print(poem_list[i]['href'])
            URL_list.append(site + poem_list[i]['href'])
            if len(URL_list) == 10:
                break
        page += 1
    return {"poet":poet, "URL":URL_list}

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
pd_title_list = ["Title", "Poem", "Poem_for_Human"]
def getPoem(file_name, URLs):
    df = pd.DataFrame()
    if os.path.exists(file_name):
        df = pd.read_csv(file_name)
    else:
        os.makedirs(os.path.dirname(file_name), exist_ok=True)
        df = pd.DataFrame(scrapePoemFromURL(URLs), columns=pd_title_list)
        df.to_csv(file_name)
    return df

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
#saving the poems of first poet Rudyard Kipling
file_name = store_loc + poet1.replace(" ", "_") + ".csv"
RP_df = getPoem(file_name, get_poem_url_json(poet1)['URL'])
#saving the poems of second poet TS Eliot
file_name = store_loc + poet2.replace(" ", "_") + ".csv"
TSE_df = getPoem(file_name, get_poem_url_json(poet2)['URL'])

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
RP_df.shape, TSE_df.shape#just confirming that each dataframe has 10 poems

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


((10, 4), (10, 4))

Creating a subdataframe that only focuses on 2 poems frome ach of the authors. (Instead of normal picking up I ll pick 2 of my favourite poems from both the poets.)

In [ ]:
RP_df['Title']

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


0                           If....
1            The Law Of The Jungle
2        The Way Through The Woods
3                            Boots
4                  The Camels Hump
5                      Fuzzy-Wuzzy
6    The Last of the Light Brigade
7        Dirge Of The Dead Sisters
8                           Sussex
9                      The Hyaenas
Name: Title, dtype: object

In [ ]:
TSE_df['Title']

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


0                     The Naming Of Cats
1                         The Hollow Men
2              Macavity: The Mystery Cat
3                         The Waste Land
4                     The Old Gumbie Cat
5    The Love Song Of J. Alfred Prufrock
6          Eyes That Last I Saw In Tears
7                The Journey Of The Magi
8                     Portrait Of A Lady
9                  Morning At The Window
Name: Title, dtype: object

In [ ]:
RP_df=RP_df.iloc[2:].reset_index(drop=True)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
RP_df

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Title,Poem,Poem_for_Human
0,The Way Through The Woods,they shut the road through the woods seventy y...,\r\n\tThey shut the road through the woods\r\n...
1,Boots,infantry columns we are foot—slog—slog—slog—sl...,INFANTRY COLUMNS\n\nWe're foot—slog—slog—slog—...
2,The Camels Hump,the camel hump is an ugly lump which well you ...,The Camel's hump is an ugly lump \nWhich well ...
3,Fuzzy-Wuzzy,we have fought with many men acrost the seas a...,"We've fought with many men acrost the seas,\n ..."
4,The Last of the Light Brigade,there were thirty million english who talked o...,There were thirty million English who talked o...
5,Dirge Of The Dead Sisters,who recalls the twilight and the ranged tents ...,Who recalls the twilight and the ranged tents ...
6,Sussex,god gave all men all earth to love but since o...,"GOD gave all men all earth to love,\n But s..."
7,The Hyaenas,after the burial parties leave and the baffled...,After the burial-parties leave\n And the baff...


In [ ]:
TSE_df=TSE_df.iloc[2:].reset_index(drop=True)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
TSE_df

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Title,Poem,Poem_for_Human
0,Macavity: The Mystery Cat,macavity mystery cat he is called the hidden p...,Macavity's a Mystery Cat: he's called the Hidd...
1,The Waste Land,part burial of the dead april is the cruelest ...,Part 1 - *Burial of the Dead*\nApril is the cr...
2,The Old Gumbie Cat,have gumbie cat in mind her name is jennyanydo...,"I have a Gumbie Cat in mind, her name is Jenny..."
3,The Love Song Of J. Alfred Prufrock,s’io credesse che mia risposta fosse persona c...,S’io credesse che mia risposta fosse \n A per...
4,Eyes That Last I Saw In Tears,eyes that last saw in tears through division h...,Eyes that last I saw in tears\r\nThrough divis...
5,The Journey Of The Magi,cold coming we had of it just the worst time o...,"A cold coming we had of it,\nJust the worst t..."
6,Portrait Of A Lady,thou hast committed— fornication but that was ...,Thou hast committed—\n Fornication: but that ...
7,Morning At The Window,they are rattling breakfast plates in basement...,They are rattling breakfast plates in basement...


## For Poet 1 we are going to be focusing on The Way Through The Woods and Boots.

## For Poet 2 we are going to be looking on the Macavity: The Mystery Cat and The Waste Land.

In [ ]:
RP_sub_df = RP_df.iloc[:2]
TSE_sub_df = TSE_df.iloc[:2]
#save both the data frame as csv
RP_sub_df.to_csv(store_loc + "RP_sub_df.csv", index=False)
TSE_sub_df.to_csv(store_loc + "TSE_sub_df.csv", index=False)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## Extracting the POS in all Poems

In [ ]:
def buildPoemArray(data):
    '''
    Converting the data into a dictionary of poems
    '''
    poet={}
    poems = list()
    for i in data:
        idx = "poem"+str(i)
        #each poem will be saved as key poem{i}
        poet[idx+"_title"] = data[i]["Title"]
        poet[idx] = data[i]["Poem"]#Line change orifnally was "Poem"
        poems.append(idx)
    poet['poems_array'] = poems
    return poet

def extractAllPOS(poet):
    '''
    Extracting the POS for each poem, the input to this function is a dictionary of poems (output of buildPoemArray)
    '''
    global_vrb = set()
    global_nns = set()
    global_adj = set()
    for key in poet['poems_array']:
        text = poet[key]
        #tokenize the words of the poem
        word_list = word_tokenize(text)
        #use pos_tag pos in each speach
        pos_value = nltk.pos_tag(word_list)
        poet["pos_"+key] = pos_value

        vrb = set([word for (word, pos) in pos_value if (pos.startswith('VB'))])
        nns = set([word for (word, pos) in pos_value if (pos.startswith('NN'))])
        adj = set([word for (word, pos) in pos_value if (pos.startswith('JJ'))])

        poet["verbs_"+key] = list(vrb)
        poet["nouns_"+key] = list(nns)
        poet["adjectives_"+key] = list(adj)

        global_vrb = set.union(global_vrb, vrb)
        global_nns = set.union(global_nns, nns)
        global_adj = set.union(global_adj, adj)

    poet["all_verbs"] = list(global_vrb)
    poet["all_nouns"] = list(global_nns)
    poet["all_adjectives"] = list(global_adj)

def getPOS(file_name, df):
    pos = ""
    if os.path.exists(file_name):
        with open(file_name, 'r') as file:
            pos = json.load(file)
    else:
        pos = buildPoemArray(json.loads(df.to_json(orient="index")))
        extractAllPOS(pos)
        pos['poet'] = file_name.split("/")[-1][:-14]
        with open(file_name, 'w') as file:
            json.dump(pos, file)
    return pos

def getJSON(file_name, df_name_1, df_1, df_name_2, df_2):
    pos_json = ""
    if os.path.exists(file_name):
        with open(file_name, 'r') as file:
            pos_json = json.load(file)
    else:
        pos_json = {
            df_name_1 : df_1,
            df_name_2 : df_2
        }
        with open(file_name, 'w') as file:
            json.dump(pos_json, file)
    return pos_json

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
#extracting the POS from all POEMS and saving them I a seperate folder
file_name = output_loc + poet1.replace(" ", "_") + 'all_poems' + ".json"
RP_pos = getPOS(file_name, RP_df)
file_name = store_loc + poet2.replace(" ", "_") + 'all_poems' + ".json"
TSE_pos = getPOS(file_name, TSE_df)
# file_name = store_loc + poet1.replace(" ", "_") + "-" + poet2.replace(" ", "_") + ".json"
# poets_json = getJSON(file_name, poet1.replace(" ", "_"), RP_pos, poet2.replace(" ", "_"), TSE_pos)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
RP_pos.keys()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


dict_keys(['poem0_title', 'poem0', 'poem1_title', 'poem1', 'poem2_title', 'poem2', 'poem3_title', 'poem3', 'poem4_title', 'poem4', 'poem5_title', 'poem5', 'poem6_title', 'poem6', 'poem7_title', 'poem7', 'poem8_title', 'poem8', 'poem9_title', 'poem9', 'poems_array', 'pos_poem0', 'verbs_poem0', 'nouns_poem0', 'adjectives_poem0', 'pos_poem1', 'verbs_poem1', 'nouns_poem1', 'adjectives_poem1', 'pos_poem2', 'verbs_poem2', 'nouns_poem2', 'adjectives_poem2', 'pos_poem3', 'verbs_poem3', 'nouns_poem3', 'adjectives_poem3', 'pos_poem4', 'verbs_poem4', 'nouns_poem4', 'adjectives_poem4', 'pos_poem5', 'verbs_poem5', 'nouns_poem5', 'adjectives_poem5', 'pos_poem6', 'verbs_poem6', 'nouns_poem6', 'adjectives_poem6', 'pos_poem7', 'verbs_poem7', 'nouns_poem7', 'adjectives_poem7', 'pos_poem8', 'verbs_poem8', 'nouns_poem8', 'adjectives_poem8', 'pos_poem9', 'verbs_poem9', 'nouns_poem9', 'adjectives_poem9', 'all_verbs', 'all_nouns', 'all_adjectives', 'poet'])

Doing the same for the sub dataframes ith only 2 poems

In [ ]:
temp = os.path.join(output_loc, 'first_2_poems')
os.makedirs(temp,exist_ok=True)
#extracting the POS for only 2 poems
file_name = temp + poet1.replace(" ", "_")  + ".json"
RP_pos_2 = getPOS(file_name, RP_sub_df)
file_name = temp + poet2.replace(" ", "_")  + ".json"
TSE_pos_2 = getPOS(file_name, TSE_sub_df)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
RP_pos_2.keys()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


dict_keys(['poem0_title', 'poem0', 'poem1_title', 'poem1', 'poems_array', 'pos_poem0', 'verbs_poem0', 'nouns_poem0', 'adjectives_poem0', 'pos_poem1', 'verbs_poem1', 'nouns_poem1', 'adjectives_poem1', 'all_verbs', 'all_nouns', 'all_adjectives', 'poet'])

# Transpose

In [ ]:
def getSemanticSimilarity(p1, p2, model):
    embedding1 = model.encode(p1, convert_to_tensor=True)
    embedding2 = model.encode(p2, convert_to_tensor=True)
    return util.pytorch_cos_sim(embedding1, embedding2).item()

def getSemanticSimilarityEmbedding(embedding1, embedding2, model):
    return util.pytorch_cos_sim(embedding1, embedding2).item()

def get_poet_embedding_dict(poet, model):
    poet_dict = dict()
    for word in poet:
      if word not in poet_dict:
        poet_dict[word] = model.encode(word, convert_to_tensor=True)
    return poet_dict

def swapWords(poet1, poet2, poem):
    model = SentenceTransformer('stsb-roberta-large')
    import time
    start = time.time()
    poet1_dict = get_poet_embedding_dict(poet1, model)
    end = time.time()
    # print(end - start)
    start = time.time()
    poet2_dict = get_poet_embedding_dict(poet2, model)
    end = time.time()
    # print(end - start)

    cs_threshold = 0.6
    new_poem = poem
    for word1 in poet1:
        max_word = ""
        max_sim = 0
        for word2 in poet2:
            if word1 == word2:
                continue
            sim = getSemanticSimilarityEmbedding(poet1_dict[word1], poet2_dict[word2], model)
            if max_sim < sim:
                max_sim = sim
                max_word = word2
        if max_sim > cs_threshold:
            new_poem = new_poem.replace(word1, max_word)
    return new_poem

def newPoem(file_name, poet_1_pos, poet_2_pos, poem, index,swap='all'):
    new_poem = ""
    if os.path.exists(file_name):
        with open(file_name, 'r') as file:
            new_poem = file.read()
    else:
        if swap == 'all':
          new_poem = swapWords(poet_1_pos['verbs_poem' + str(index)], poet_2_pos['all_verbs'], poem[index].lower())
          new_poem = swapWords(poet_1_pos['adjectives_poem'+ str(index)], poet_2_pos['all_adjectives'], new_poem)
          new_poem = swapWords(poet_1_pos['nouns_poem'+ str(index)], poet_2_pos['all_nouns'], new_poem)
        elif swap  =='verb':
          new_poem = swapWords(poet_1_pos['verbs_poem' + str(index)], poet_2_pos['all_verbs'], poem[index].lower())
        elif swap == 'adjective':
          new_poem = swapWords(poet_1_pos['adjectives_poem'+ str(index)], poet_2_pos['all_adjectives'], poem[index].lower())
        elif swap == 'noun':
          new_poem = swapWords(poet_1_pos['nouns_poem'+ str(index)], poet_2_pos['all_nouns'], poem[index].lower())
        with open(file_name, 'w') as file:
            file.write(new_poem)
    return new_poem

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## Swapping Poets 1's POS with Poet 2 and Comparing the Similarities

In [ ]:
def swap_operations( poet1, poet2, poet1_df, poet2_df, poet1_pos, poet2_pos, poem_idx,pos_used='pos_2_poem'):
    assert pos_used in ['pos_2_poem', 'pos_10_poem']
    swap = ['verb','adjective','noun', 'all']
    poems = {}
    poems['og'] = poet1_df.Poem_for_Human[poem_idx]
    #similarity between the Poets 1's poem_idx and Poet 2's poem_idx poem
    print(f"Before Swapping the similarty between {poet1} poem and {poet2} poem")
    print(getSemanticSimilarity(poet1_df.Poem[poem_idx], poet2_df.Poem[poem_idx], SentenceTransformer('stsb-roberta-large')))
    store_loc = f'/content/drive/MyDrive/ML-Spring-2024/deep-poets/Team-Pi/swaps/{pos_used}/'
    poem_name = poet1_df.iloc[poem_idx].Title
    poem_name = poem_name.replace('\'','').replace('-','').replace(',','').replace(':','').replace('  ', ' ').replace(' ', '_')
    os.makedirs(store_loc,exist_ok=True)
    for op in swap:
      #swap poets 1 poem with poet 2's pos, based  swap  value
      #base_path /content/drive/MyDrive/ML-Spring-2024/deep-poets/Team-Pi/swaps/
      if op != 'all':
        file_name = store_loc + poet1.replace(" ", "_") + '-' + poet2.replace(" ", "_") + '-' + poem_name + '-' + op + '.txt'

      else:
        file_name = store_loc + poet1.replace(" ", "_") + '-' + poet2.replace(" ", "_") + '-' + poem_name + '.txt'

      new = newPoem(file_name, poet1_pos, poet2_pos, poet1_df.Poem_for_Human, poem_idx, op)
      poems[op] = new
      #compare the similairty of new poem with Swap, with second poets poem_idx poem
      print(f'After Swapping {op} on poem {poem_name}, the similarty between the new and old poem is')
      print(getSemanticSimilarity(new, poet1_df.Poem[poem_idx], SentenceTransformer('stsb-roberta-large')))

    print(f'After Swaping all, the similaritie between {poem_name} written by {poet1} and {poet2}')
    print(getSemanticSimilarity(poet1_df.Poem[poem_idx], poems['all'], SentenceTransformer('stsb-roberta-large')))
    return poems


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


#### Poem 1 by Author 1

##### Using Part of Speech Identified from 2 Poems

In [ ]:
print('Performing Various POS swaps on Poem 1 of Poet 1, using POS generated from only 2 poem ')
poems0_RP_TSE_sub=swap_operations(poet1, poet2, RP_sub_df, TSE_sub_df, RP_pos_2, TSE_pos_2, 0)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Performing Various POS swaps on Poem 1 of Poet 1, using POS generated from only 2 poem 
Before Swapping the similarty between Rudyard Kipling poem and T S Eliot poem


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.96k [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

0.46410346031188965
After Swapping verb on poem The_Way_Through_The_Woods, the similarty between the new and old poem is
0.9999999403953552
After Swapping adjective on poem The_Way_Through_The_Woods, the similarty between the new and old poem is
0.9999999403953552
After Swapping noun on poem The_Way_Through_The_Woods, the similarty between the new and old poem is
0.9986393451690674
After Swapping all on poem The_Way_Through_The_Woods, the similarty between the new and old poem is
0.9986393451690674
After Swaping all, the similaritie between The_Way_Through_The_Woods written by Rudyard Kipling and T S Eliot
0.9986393451690674


In [ ]:
print('Old Poem')
print(poems0_RP_TSE_sub['og'])
print('\n')
print('New Poem')
print(poems0_RP_TSE_sub['all'])

Old Poem

	They shut the road through the woods
	Seventy years ago.
	Weather and rain have undone it again,
	And now you would never know
	There was once a road through the woods
	Before they planted the trees.
	It is underneath the coppice and heath,
	And the thin anemones.
	Only the keeper sees
	That, where the ring-dove broods,
	And the badgers roll at ease,
	There was once a road through the woods.
	Yet, if you enter the woods
	Of a summer evening late,
	When the night-air cools on the trout-ringed pools
	Where the otter whistles his mate.
	(They fear not men in the woods,
	Because they see so few)
	You will hear the beat of a horse's feet,
	And the swish of a skirt in the dew,
	Steadily cantering through
	The misty solitudes,
	As though they perfectly knew
	The old lost road through the woods….
	But there is no road through the woods.
	Line 20 There has seen much debate about the meaning of "skirt" at this point with 3 basic meanings predominant.

	either

	 a 'skirt' is the hairy

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


##### Using Part of Speech Identified from 10 Poems

In [ ]:
print('Performing Various POS swaps on Poem 1 of Poet 1, using POS generated all 10 poems ')
poems0_RP_TSE=swap_operations(poet1, poet2, RP_df, TSE_df, RP_pos, TSE_pos, 0, pos_used='pos_10_poem')

Performing Various POS swaps on Poem 1 of Poet 1, using POS generated all 10 poems 
Before Swapping the similarty between Rudyard Kipling poem and T S Eliot poem


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


0.46410346031188965
After Swapping verb on poem The_Way_Through_The_Woods, the similarty between the new and old poem is
0.8902893662452698
After Swapping adjective on poem The_Way_Through_The_Woods, the similarty between the new and old poem is
0.9999999403953552
After Swapping noun on poem The_Way_Through_The_Woods, the similarty between the new and old poem is
0.9950220584869385
After Swapping all on poem The_Way_Through_The_Woods, the similarty between the new and old poem is
0.8750931024551392
After Swaping all, the similaritie between The_Way_Through_The_Woods written by Rudyard Kipling and T S Eliot
0.8750931024551392


In [ ]:
print('Old Poem')
print(poems0_RP_TSE['og'])
print('\n')
print('New Poem')
print(poems0_RP_TSE['all'])

Old Poem

	They shut the road through the woods
	Seventy years ago.
	Weather and rain have undone it again,
	And now you would never know
	There was once a road through the woods
	Before they planted the trees.
	It is underneath the coppice and heath,
	And the thin anemones.
	Only the keeper sees
	That, where the ring-dove broods,
	And the badgers roll at ease,
	There was once a road through the woods.
	Yet, if you enter the woods
	Of a summer evening late,
	When the night-air cools on the trout-ringed pools
	Where the otter whistles his mate.
	(They fear not men in the woods,
	Because they see so few)
	You will hear the beat of a horse's feet,
	And the swish of a skirt in the dew,
	Steadily cantering through
	The misty solitudes,
	As though they perfectly knew
	The old lost road through the woods….
	But there is no road through the woods.
	Line 20 There has seen much debate about the meaning of "skirt" at this point with 3 basic meanings predominant.

	either

	 a 'skirt' is the hairy

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


#### Poem 2 By Poet 1

##### Using Part of Speech Identified from 2 Poems

In [ ]:
print('Performing Various POS swaps on Poem 2 of Poet 1, using POS generated from only 2 poem ')
poems1_RP_TSE_sub=swap_operations(poet1, poet2, RP_sub_df, TSE_sub_df, RP_pos_2, TSE_pos_2, 1)

Performing Various POS swaps on Poem 2 of Poet 1, using POS generated from only 2 poem 
Before Swapping the similarty between Rudyard Kipling poem and T S Eliot poem


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


0.20202192664146423
After Swapping verb on poem Boots, the similarty between the new and old poem is
0.9263428449630737
After Swapping adjective on poem Boots, the similarty between the new and old poem is
0.9999998807907104
After Swapping noun on poem Boots, the similarty between the new and old poem is
0.9686354398727417
After Swapping all on poem Boots, the similarty between the new and old poem is
0.9005382061004639
After Swaping all, the similaritie between Boots written by Rudyard Kipling and T S Eliot
0.9005382061004639


In [ ]:
print('Old Poem')
print(poems1_RP_TSE_sub['og'])
print('\n')
print('New Poem')
print(poems1_RP_TSE_sub['all'])

Old Poem
INFANTRY COLUMNS

We're foot—slog—slog—slog—sloggin' over Africa —
Foot—foot—foot—foot—sloggin' over Africa —
(Boots—boots—boots—boots—movin' up an' down again!)
                There's no discharge in the war!

Seven—six—eleven—five—nine-an'-twenty mile to-day —
Four—eleven—seventeen—thirty-two the day before —
(Boots—boots—boots—boots—movin' up an' down again!)
                There's no discharge in the war!

Don't—don't—don't—don't—look at what's in front of you.
(Boots—boots—boots—boots—movin' up an' down again);
Men—men—men—men—men go mad with watchin' em,
                An' there's no discharge in the war!

Try—try—try—try—to think o' something different —
Oh—my—God—keep—me from goin' lunatic!
(Boots—boots—boots—boots—movin' up an' down again!)
                There's no discharge in the war!

Count—count—count—count—the bullets in the bandoliers.
If—your—eyes—drop—they will get atop o' you!
(Boots—boots—boots—boots—movin' up an' down again) —
                There's n

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


##### Using Part of Speech Identified from 10 Poems

In [ ]:
print('Performing Various POS swaps on Poem 2 of Poet 1, using POS generated all 10 poems ')
poems1_RP_TSE=swap_operations(poet1, poet2, RP_df, TSE_df, RP_pos, TSE_pos, 1, pos_used='pos_10_poem')

Performing Various POS swaps on Poem 2 of Poet 1, using POS generated all 10 poems 
Before Swapping the similarty between Rudyard Kipling poem and T S Eliot poem


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


0.20202192664146423
After Swapping verb on poem Boots, the similarty between the new and old poem is
0.8679007291793823
After Swapping adjective on poem Boots, the similarty between the new and old poem is
0.984450101852417
After Swapping noun on poem Boots, the similarty between the new and old poem is
0.9977989792823792
After Swapping all on poem Boots, the similarty between the new and old poem is
0.8665899634361267
After Swaping all, the similaritie between Boots written by Rudyard Kipling and T S Eliot
0.8665899634361267


In [ ]:
print('Old Poem')
print(poems1_RP_TSE['og'])
print('\n')
print('New Poem')
print(poems1_RP_TSE['all'])

Old Poem
INFANTRY COLUMNS

We're foot—slog—slog—slog—sloggin' over Africa —
Foot—foot—foot—foot—sloggin' over Africa —
(Boots—boots—boots—boots—movin' up an' down again!)
                There's no discharge in the war!

Seven—six—eleven—five—nine-an'-twenty mile to-day —
Four—eleven—seventeen—thirty-two the day before —
(Boots—boots—boots—boots—movin' up an' down again!)
                There's no discharge in the war!

Don't—don't—don't—don't—look at what's in front of you.
(Boots—boots—boots—boots—movin' up an' down again);
Men—men—men—men—men go mad with watchin' em,
                An' there's no discharge in the war!

Try—try—try—try—to think o' something different —
Oh—my—God—keep—me from goin' lunatic!
(Boots—boots—boots—boots—movin' up an' down again!)
                There's no discharge in the war!

Count—count—count—count—the bullets in the bandoliers.
If—your—eyes—drop—they will get atop o' you!
(Boots—boots—boots—boots—movin' up an' down again) —
                There's n

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


#### Take Away
Replaced with POS Identified from 2 Poems of Poet 2 (T.S. Eliot):

This version introduces mild disruptions to the original structure and vocabulary. The words swapped in are mostly congruent in their part of speech but differ significantly in their connotations or usual contexts. This swap results in subtle shifts in meaning and a slightly more disjointed feel, although the underlying structure of Kipling's poem is still recognizable.

Replaced with POS Identified from 10 Poems of Poet 2:

This version shows more pronounced disruptions. The increase in the number of poems used to identify POS tags might have introduced a broader vocabulary, but this also leads to greater semantic and syntactic irregularities. The text often becomes grammatically inconsistent. Such changes make the poem harder to follow and diminish its poetic flow and impact.

## Swapping Poets 2's POS with Poet 1 and Comparing the Similarities

#### Poem 1 By Poet 2

##### Using Part of Speech Identified from 2 Poems

In [ ]:
print('Performing Various POS swaps on Poem 2 of Poet 2, using POS generated from only 2 poem ')
poems1_TSE_RP_sub=swap_operations(poet2, poet1, TSE_sub_df, RP_sub_df, TSE_pos_2, RP_pos_2, 1)

Performing Various POS swaps on Poem 2 of Poet 2, using POS generated from only 2 poem 
Before Swapping the similarty between T S Eliot poem and Rudyard Kipling poem


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


0.20202192664146423
After Swapping verb on poem The_Waste_Land, the similarty between the new and old poem is
0.9575403928756714
After Swapping adjective on poem The_Waste_Land, the similarty between the new and old poem is
0.9983029365539551
After Swapping noun on poem The_Waste_Land, the similarty between the new and old poem is
0.9999998807907104
After Swapping all on poem The_Waste_Land, the similarty between the new and old poem is
0.9559998512268066
After Swaping all, the similaritie between The_Waste_Land written by T S Eliot and Rudyard Kipling
0.9559998512268066


In [ ]:
print('Old Poem')
print(poems1_TSE_RP_sub['og'])
print('\n')
print('New Poem')
print(poems1_TSE_RP_sub['all'])

Old Poem
Part 1 - *Burial of the Dead*
April is the cruelest month, breeding
Lilacs out of the dead land, mixing
Memory and desire, stirring
Dull roots with spring rain.
Winter kept us warm, covering
Earth in forgetful snow, feeding
A little life with dried tubers.
Summer surprised us, coming over the Starnbergersee
With a shower of rain; we stopped in the colonnade,
And went on in sunlight, into the Hofgarten
And drank coffee, and talked for an hour.
Bin gar keine Russin, stamm' aus Litauen, echt deutsch.
And when we were children, staying at the arch-duke's,
My cousin's, he took me out on a sled,
And I was frightened. He said, Marie,
Marie, hold on tight.  And down we went.
In the mountains, there you feel free.
I read, much of the night, and go south in the winter.

What are the roots that clutch, what branches grow
Out of this stony rubbish?  Son of man,
You canot say, or guess, for you know only
A heap of broken images, where the sun beats,
And the dead tree gives no shelter, the 

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


##### Using Part of Speech Identified from 10 Poems

In [ ]:
print('Performing Various POS swaps on Poem 2 of Poet 2, using POS generated all 10 poems  ')
poems1_TSE_RP = swap_operations(poet2, poet1, TSE_sub_df, RP_sub_df, TSE_pos, RP_pos, 1, pos_used='pos_10_poem')

Performing Various POS swaps on Poem 2 of Poet 2, using POS generated all 10 poems  
Before Swapping the similarty between T S Eliot poem and Rudyard Kipling poem


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


0.20202192664146423
After Swapping verb on poem The_Waste_Land, the similarty between the new and old poem is
0.9759970903396606
After Swapping adjective on poem The_Waste_Land, the similarty between the new and old poem is
0.9992699027061462
After Swapping noun on poem The_Waste_Land, the similarty between the new and old poem is
0.9999080896377563
After Swapping all on poem The_Waste_Land, the similarty between the new and old poem is
0.9730604887008667
After Swaping all, the similaritie between The_Waste_Land written by T S Eliot and Rudyard Kipling
0.9730604887008667


In [ ]:
print('Old Poem')
print(poems1_TSE_RP['og'])
print('\n')
print('New Poem')
print(poems1_TSE_RP['all'])

Old Poem
Part 1 - *Burial of the Dead*
April is the cruelest month, breeding
Lilacs out of the dead land, mixing
Memory and desire, stirring
Dull roots with spring rain.
Winter kept us warm, covering
Earth in forgetful snow, feeding
A little life with dried tubers.
Summer surprised us, coming over the Starnbergersee
With a shower of rain; we stopped in the colonnade,
And went on in sunlight, into the Hofgarten
And drank coffee, and talked for an hour.
Bin gar keine Russin, stamm' aus Litauen, echt deutsch.
And when we were children, staying at the arch-duke's,
My cousin's, he took me out on a sled,
And I was frightened. He said, Marie,
Marie, hold on tight.  And down we went.
In the mountains, there you feel free.
I read, much of the night, and go south in the winter.

What are the roots that clutch, what branches grow
Out of this stony rubbish?  Son of man,
You canot say, or guess, for you know only
A heap of broken images, where the sun beats,
And the dead tree gives no shelter, the 

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


#### Poem 2 By Poet 2

##### Using Part of Speech Identified from 2 Poems

In [ ]:
print('Performing Various POS swaps on Poem 1 of Poet 2, using POS generated from only 2 poem ')
poems0_TSE_RP_sub=swap_operations(poet2, poet1, TSE_sub_df, RP_sub_df, TSE_pos_2, RP_pos_2, 0)

Performing Various POS swaps on Poem 1 of Poet 2, using POS generated from only 2 poem 
Before Swapping the similarty between T S Eliot poem and Rudyard Kipling poem


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


0.46410346031188965
After Swapping verb on poem Macavity_The_Mystery_Cat, the similarty between the new and old poem is
0.9631263613700867
After Swapping adjective on poem Macavity_The_Mystery_Cat, the similarty between the new and old poem is
1.0
After Swapping noun on poem Macavity_The_Mystery_Cat, the similarty between the new and old poem is
0.9940515160560608
After Swapping all on poem Macavity_The_Mystery_Cat, the similarty between the new and old poem is
0.9631263613700867
After Swaping all, the similaritie between Macavity_The_Mystery_Cat written by T S Eliot and Rudyard Kipling
0.9631263613700867


In [ ]:
print('Old Poem')
print(poems0_TSE_RP_sub['og'])
print('\n')
print('New Poem')
print(poems0_TSE_RP_sub['all'])

Old Poem
Macavity's a Mystery Cat: he's called the Hidden Paw—
For he's the master criminal who can defy the Law.
He's the bafflement of Scotland Yard, the Flying Squad's despair:
For when they reach the scene of crime—Macavity's not there!

Macavity, Macavity, there's no one like Macavity,
He's broken every human law, he breaks the law of gravity.
His powers of levitation would make a fakir stare,
And when you reach the scene of crime—Macavity's not there!
You may seek him in the basement, you may look up in the air—
But I tell you once and once again, Macavity's not there!

Macavity's a ginger cat, he's very tall and thin;
You would know him if you saw him, for his eyes are sunken in.
His brow is deeply lined with thought, his head is highly domed;
His coat is dusty from neglect, his whiskers are uncombed.
He sways his head from side to side, with movements like a snake;
And when you think he's half asleep, he's always wide awake.

Macavity, Macavity, there's no one like Macavity,
Fo

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


##### Using Part of Speech Identified from 10 Poems

In [ ]:
print('Performing Various POS swaps on Poem 1 of Poet 2, using POS generated all 10 poems  ')
poems0_TSE_RP = swap_operations(poet2, poet1, TSE_sub_df, RP_sub_df, TSE_pos, RP_pos, 0, pos_used='pos_10_poem')

Performing Various POS swaps on Poem 1 of Poet 2, using POS generated all 10 poems  
Before Swapping the similarty between T S Eliot poem and Rudyard Kipling poem


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


0.46410346031188965
After Swapping verb on poem Macavity_The_Mystery_Cat, the similarty between the new and old poem is
0.9675309658050537
After Swapping adjective on poem Macavity_The_Mystery_Cat, the similarty between the new and old poem is
1.0
After Swapping noun on poem Macavity_The_Mystery_Cat, the similarty between the new and old poem is
0.9851032495498657
After Swapping all on poem Macavity_The_Mystery_Cat, the similarty between the new and old poem is
0.9675309658050537
After Swaping all, the similaritie between Macavity_The_Mystery_Cat written by T S Eliot and Rudyard Kipling
0.9675309658050537


In [ ]:
print('Old Poem')
print(poems0_TSE_RP['og'])
print('\n')
print('New Poem')
print(poems0_TSE_RP['all'])

Old Poem
Macavity's a Mystery Cat: he's called the Hidden Paw—
For he's the master criminal who can defy the Law.
He's the bafflement of Scotland Yard, the Flying Squad's despair:
For when they reach the scene of crime—Macavity's not there!

Macavity, Macavity, there's no one like Macavity,
He's broken every human law, he breaks the law of gravity.
His powers of levitation would make a fakir stare,
And when you reach the scene of crime—Macavity's not there!
You may seek him in the basement, you may look up in the air—
But I tell you once and once again, Macavity's not there!

Macavity's a ginger cat, he's very tall and thin;
You would know him if you saw him, for his eyes are sunken in.
His brow is deeply lined with thought, his head is highly domed;
His coat is dusty from neglect, his whiskers are uncombed.
He sways his head from side to side, with movements like a snake;
And when you think he's half asleep, he's always wide awake.

Macavity, Macavity, there's no one like Macavity,
Fo

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


###Take Away:
Replaced with POS Identified from 2 Poems:

This version introduces a variety of grammatical errors and mismatches that result in nonsensical phrases. These errors disrupt the poem's original flow and elegance.
The substitutions made with a smaller dataset (2 poems) appear to inconsistently match the part of speech, leading to erratic shifts in vocabulary that occasionally maintain the poem's rhyme but often distort the meaning and readability.


Replaced with POS Identified from 10 Poems:

The increased dataset leads to somewhat better lexical variety, but similar issues with grammar and cohesion persist. The language becomes slightly more varied, yet this doesn't necessarily enhance clarity or poetic quality.
Phrases like "names that never hearted to much than one cat" show an attempt to maintain the original poem's structure but fail in coherence and grammatical accuracy.

# Topic Modeling and Summarization

## Summarization

In [ ]:
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
summarizer = pipeline("summarization", model="t5-base", tokenizer="t5-base", framework="tf")

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFT5ForConditionalGeneration.

All the weights of TFT5ForConditionalGeneration were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

In [ ]:
def removeQuotes(mydata):
    new_data = mydata.strip('“”')
    return new_data

def SummaryOfPoem(poet):
  count = 0
  for key in poet['poems_array']:
    count += 1;
    text = poet[key]
    text = removeQuotes(text)
    summary_text = summarizer(text, max_length=50, min_length=5, do_sample=False)[0]['summary_text']
    poet["summary_"+key] = summary_text

  return poet

def getSummary(file_name, df):
    summary = ""
    if os.path.exists(file_name):
        with open(file_name, 'r') as file:
            summary = json.load(file)
    else:
        summary = SummaryOfPoem(buildPoemArray(json.loads(df.to_json(orient="index"))))
        with open(file_name, 'w') as file:
            json.dump(summary, file)
    return summary

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
file_name = output_loc + poet1.replace(" ", "_") + "-Summary.json"
RP_summary = getSummary(file_name, RP_df)
file_name = output_loc + poet2.replace(" ", "_") + "-Summary.json"
TSE_summary = getSummary(file_name, TSE_df)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
print(poet1)
for k in RP_summary.keys():
  if k[:7]=='summary':
    print(k)
    print(RP_summary[k])
print('\n')
print(poet2)
for k in TSE_summary.keys():
  if k[:7]=='summary':
    print(k)
    print(TSE_summary[k])

Rudyard Kipling
summary_poem0
if you can make one heap of all your winnings and risk it on one turn of pitch and toss and lose and start again at your beginnings and never breathe word about your loss .
summary_poem1
the wolf that shall keep it may prosper but he that shall break it must die . the law of the jungle is many and mighty but the head and the hoof of the law and the haunch and the hum
summary_poem2
there is no road through the woods line 20 there has seen much debate about the meaning of skirt at this point with basic meanings predominant either skirt is the hairy part of horse tail or skirt is an archaic word for part of
summary_poem3
faber music ltd in england and is apparently available for male chorus by peter dawson mccall . boots by the great american baritone leonard warren is included in le
summary_poem4
the camel hump is an ugly lump which you may see at the zoo . but uglier yet is the lump we get from having too little to do kiddies and grown ups too oo
summary_po

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


#### Take away
 Some capture the essence of Kipling’s and Eliot's work effectively, while others suffer from lack of clarity, context, or accuracy.

## Topic Modelling

### LDA

In [ ]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))

def choherence_perplexity_calculator(poem_list):
  data_words = list(sent_to_words(poem_list))
  print(data_words[:1])
  bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100)
  trigram = gensim.models.Phrases(bigram[data_words], threshold=100)

  bigram_mod = gensim.models.phrases.Phraser(bigram)
  trigram_mod = gensim.models.phrases.Phraser(trigram)

  print(trigram_mod[bigram_mod[data_words[0]]])


  def remove_stopwords(texts):
      return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

  def make_bigrams(texts):
      return [bigram_mod[doc] for doc in texts]

  def make_trigrams(texts):
      return [trigram_mod[bigram_mod[doc]] for doc in texts]

  def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
      """https://spacy.io/api/annotation"""
      texts_out = []
      for sent in texts:
          doc = nlp(" ".join(sent))
          texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
      return texts_out
  # Remove Stop Words
  #data_words_nostops = remove_stopwords(data_words)

  # Form Bigrams
  data_words_bigrams = make_bigrams(data_words)

  # Initialize spacy 'en' model, keeping only tagger component (for efficiency)
  # python3 -m spacy download en
  nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

  # Do lemmatization keeping only noun, adj, vb, adv
  data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN']) #, 'ADJ', 'VERB', 'ADV'

  print(data_lemmatized[:1])
  # Create Dictionary
  id2word = corpora.Dictionary(data_lemmatized)

  # Create Corpus
  texts = data_lemmatized

  # Term Document Frequency
  corpus = [id2word.doc2bow(text) for text in texts]

  # View
  print(corpus[:1])

  lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=10,
                                       random_state=100,
                                       chunksize=100,
                                       passes=10,
                                       per_word_topics=True)
  print(lda_model.print_topics())
  # Visualize the topics
  pyLDAvis.enable_notebook()
  vis = gensimvis.prepare(lda_model, corpus, id2word)
  # vis
  # Compute Perplexity
  print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

  # Compute Coherence Score
  coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
  coherence_lda = coherence_model_lda.get_coherence()
  print('\nCoherence Score: ', coherence_lda)
  #return vis
  return lda_model, corpus, id2word


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
RP_input = json.loads(RP_df.to_json(orient="index"))
TSE_input = json.loads(TSE_df.to_json(orient="index"))

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
poem_count = len(RP_input)
RP_list=[]
for j in RP_input:
  poem_index =  str(j)
  RP_input[poem_index].pop('Poem_for_Human')
  RP_list.append(RP_input[poem_index])
print(RP_list)

[{'Title': 'The Way Through The Woods', 'Poem': 'they shut the road through the woods seventy years ago weather and rain have undone it again and now you would never know there was once road through the woods before they planted the trees it is underneath the coppice and heath and the thin anemones only the keeper sees that where the ring dove broods and the badgers roll at ease there was once road through the woods yet if you enter the woods of summer evening late when the night air cools on the trout ringed pools where the otter whistles his mate they fear not men in the woods because they see so few you will hear the beat of horse feet and the swish of skirt in the dew steadily cantering through the misty solitudes as though they perfectly knew the old lost road through the woods… but there is no road through the woods line 20 there has seen much debate about the meaning of skirt at this point with basic meanings predominant either skirt is the hairy part of horse tail or skirt is a

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
poem_count = len(TSE_input)
TSE_list=[]
for j in TSE_input:
  poem_index =  str(j)
  TSE_input[poem_index].pop('Poem_for_Human')
  TSE_list.append(TSE_input[poem_index])
print(TSE_list)

[{'Title': 'Macavity: The Mystery Cat', 'Poem': 'macavity mystery cat he is called the hidden paw— for he is the master criminal who can defy the law he is the bafflement of scotland yard the flying squad despair for when they reach the scene of crime—macavity not there macavity macavity there is no one like macavity he is broken every human law he breaks the law of gravity his powers of levitation would make fakir stare and when you reach the scene of crime—macavity not there you may seek him in the basement you may look up in the air— but tell you once and once again macavity not there macavity ginger cat he is very tall and thin you would know him if you saw him for his eyes are sunken in his brow is deeply lined with thought his head is highly domed his coat is dusty from neglect his whiskers are uncombed he sways his head from side to side with movements like snake and when you think he is half asleep he is always wide awake macavity macavity there is no one like macavity for he i

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
lda_model, bow, diction = choherence_perplexity_calculator(RP_list)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[['title', 'the', 'way', 'through', 'the', 'woods', 'poem', 'they', 'shut', 'the', 'road', 'through', 'the', 'woods', 'seventy', 'years', 'ago', 'weather', 'and', 'rain', 'have', 'undone', 'it', 'again', 'and', 'now', 'you', 'would', 'never', 'know', 'there', 'was', 'once', 'road', 'through', 'the', 'woods', 'before', 'they', 'planted', 'the', 'trees', 'it', 'is', 'underneath', 'the', 'coppice', 'and', 'heath', 'and', 'the', 'thin', 'anemones', 'only', 'the', 'keeper', 'sees', 'that', 'where', 'the', 'ring', 'dove', 'broods', 'and', 'the', 'badgers', 'roll', 'at', 'ease', 'there', 'was', 'once', 'road', 'through', 'the', 'woods', 'yet', 'if', 'you', 'enter', 'the', 'woods', 'of', 'summer', 'evening', 'late', 'when', 'the', 'night', 'air', 'cools', 'on', 'the', 'trout', 'ringed', 'pools', 'where', 'the', 'otter', 'whistles', 'his', 'mate', 'they', 'fear', 'not', 'men', 'in', 'the', 'woods', 'because', 'they', 'see', 'so', 'few', 'you', 'will', 'hear', 'the', 'beat', 'of', 'horse', 'feet

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


[(0, '0.037*"man" + 0.015*"ome" + 0.015*"formation" + 0.015*"fightin" + 0.015*"paper" + 0.015*"class" + 0.008*"boundin" + 0.008*"sentry" + 0.008*"second" + 0.008*"hadendowah"'), (1, '0.036*"brigade" + 0.018*"man" + 0.018*"food" + 0.018*"sir" + 0.018*"child" + 0.018*"thing" + 0.012*"night" + 0.012*"file" + 0.012*"singer" + 0.012*"soldier"'), (2, '0.021*"man" + 0.021*"ground" + 0.021*"sea" + 0.021*"heart" + 0.021*"sussex" + 0.016*"earth" + 0.011*"land" + 0.011*"clay" + 0.011*"spot" + 0.011*"belt"'), (3, '0.068*"wood" + 0.043*"road" + 0.043*"skirt" + 0.026*"poem" + 0.026*"meaning" + 0.018*"point" + 0.018*"part" + 0.018*"horse" + 0.009*"man" + 0.009*"sister"'), (4, '0.002*"man" + 0.002*"sea" + 0.002*"ground" + 0.002*"sussex" + 0.002*"heart" + 0.002*"earth" + 0.002*"bell" + 0.002*"belt" + 0.002*"clay" + 0.002*"love"'), (5, '0.004*"boot" + 0.003*"discharge" + 0.003*"man" + 0.003*"war" + 0.002*"foot" + 0.002*"day" + 0.002*"count" + 0.002*"song" + 0.002*"down" + 0.002*"poetry"'), (6, '0.226*"b

/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/backend/fork_exec.py:38: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid = os.fork()



Perplexity:  -6.033857820252346

Coherence Score:  0.8371921706583505


In [ ]:
gensimvis.prepare(lda_model, bow, diction)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2      0.074125 -0.180072       1        1  21.481854
1      0.041145  0.170996       2        1  17.998238
6     -0.245570 -0.009863       3        1  16.104280
7      0.069946  0.043022       4        1  14.360245
0     -0.018004 -0.023372       5        1  13.488244
3      0.045939  0.011882       6        1  10.872332
8     -0.010391 -0.003890       7        1   5.350987
5      0.010172 -0.002643       8        1   0.114607
4      0.016478 -0.003735       9        1   0.114607
9      0.016160 -0.002327      10        1   0.114607, topic_info=          Term       Freq      Total Category  logprob  loglift
56        boot  25.000000  25.000000  Default  30.0000  30.0000
107       hump   5.000000   5.000000  Default  29.0000  29.0000
50        wood   6.000000   6.000000  Default  28.0000  28.0000
94         war   6.000000   6.000000  Default  27.0000  27.0000
68   discharge   6.000000   6.000000  Default  26.0000  26.0000
..         ...        ...        ...      ...      ...      ...
0          air   0.001839   2.485740  Topic10  -6.0608  -0.4375
52        year   0.001839   2.485803  Topic10  -6.0608  -0.4376
206       hell   0.001839   1.903789  Topic10  -6.0609  -0.1710
277       hand   0.001838   1.903335  Topic10  -6.0613  -0.1712
232       soul   0.001836   2.676755  Topic10  -6.0624  -0.5132

[546 rows x 6 columns], token_table=      Topic      Freq   Term
term                        
244       4  0.854476  abode
246       4  0.854369  agony
0         4  0.402295    air
0         5  0.402295    air
0         6  0.402295    air
...     ...       ...    ...
412       1  0.990059    yea
52        4  0.402285   year
52        5  0.402285   year
52        6  0.402285   year
413       1  0.801621  yearn

[354 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 2, 7, 8, 1, 4, 9, 6, 5, 10])

In [ ]:
lda_model.top_topics(bow)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[([(0.037312042, 'man'),
   (0.015375843, 'ome'),
   (0.015372431, 'formation'),
   (0.015368077, 'fightin'),
   (0.015364798, 'paper'),
   (0.015345564, 'class'),
   (0.008061408, 'boundin'),
   (0.008058145, 'sentry'),
   (0.008057865, 'second'),
   (0.00805727, 'hadendowah'),
   (0.008056282, 'shammin'),
   (0.008056204, 'reward'),
   (0.008055743, 'force'),
   (0.008055158, 'odd'),
   (0.008054734, 'square'),
   (0.0080545675, 'kid'),
   (0.008054397, 'style'),
   (0.008054195, 'cat'),
   (0.008053861, 'bayonet'),
   (0.008053683, 'rubber')],
  -0.17917594691480548),
 ([(0.03575153, 'sister'),
   (0.028757889, 'face'),
   (0.02174438, 'body'),
   (0.014721251, 'dust'),
   (0.0077218153, 'uitvlugt'),
   (0.0077199615, 'battle'),
   (0.007719674, 'clay'),
   (0.0077193035, 'name'),
   (0.0077191875, 'dirge'),
   (0.0077187, 'night'),
   (0.0077186134, 'autumn'),
   (0.007718489, 'hair'),
   (0.0077184644, 'title'),
   (0.0077182553, 'coach'),
   (0.007718251, 'peak'),
   (0.007717711

###Take Away

Topic with 'man', 'formation', 'fightin', etc.:
This cluster, featuring terms like "man," "formation," "fightin," and "sentry," likely points to themes of military life and discipline. The presence of "bayonet" and "rubber" could hint at physical aspects of military training or combat. Conclusion: This topic encapsulates the regimented and structured environment of military settings, possibly reflecting on the strictures and the personal struggles within such confines.

Topic with 'sister', 'face', 'body', etc.:
Words such as "sister," "face," "body," "dust," and "dirge" suggest themes of family, mortality, and remembrance. "Autumn," "clay," and "dust" may metaphorically represent decay or the end of life cycles. Conclusion: This cluster reflects on familial ties and the inevitable passage of time, possibly exploring the emotional depth of personal loss and the natural process of aging or death.

Topic with 'hump', 'camel', 'boot', etc.:
This cluster includes a mix of words like "hump," "camel," and "boot," combining elements of a fable or story (like "How the Camel Got His Hump") with more grounded, everyday items like "boot." "Garden," "sun," and "fire" add a touch of either domesticity or myth. Conclusion: This topic might juxtapose the fantastical or narrative-driven elements with the mundane, exploring how stories intertwine with everyday realities.

Topic with 'man', 'ground', 'sea', etc.:
Here, natural elements like "ground," "sea," "earth," and "clay" are prominent, paired with "man," "heart," and "love." This suggests themes of nature's expanse and human emotions tied to it. Conclusion:

This topic likely reflects on human connections to the earth and the sea, perhaps touching on themes of belonging, exploration, or environmental consciousness. It could be exploring how humans are inextricably linked to and affected by the natural world.

Topic with 'brigade', 'man', 'food', etc.:
With a focus on words like "brigade," "man," "food," and military terms like "troop," "charge," and "sergeant," this cluster strongly emphasizes aspects of military organization and the camaraderie or challenges within. Conclusion: This topic likely delves into the collective and personal experiences of soldiers, emphasizing the logistical and emotional aspects of military life, possibly exploring themes of brotherhood, hierarchy, and survival.

Topic with 'wood', 'road', 'skirt', etc.:
This cluster, including "wood," "road," "skirt," along with "poem" and "meaning," suggests a setting of travel or a journey through nature, intertwined with poetic reflection or storytelling. Conclusion: This topic might represent a metaphorical or literal journey through life or nature, using the setting as a backdrop for deeper philosophical or existential musings.

Topic with 'boot', 'war', 'discharge', etc.:
This cluster is clearly focused on the military with terms like "boot," "war," and "discharge." The inclusion of "poetry," "music," and "song" within this military context suggests a blend of harsh realities with the artistic expressions that often comment on or arise from them. Conclusion: Reflecting on the duality of war's grim realities and the cathartic or reflective art it inspires, this topic may highlight how conflict influences culture and personal identity.

Topic with 'sister', 'face', 'body', etc.:
This cluster reiterates themes of personal identity and human conditions with a focus on body-related terms and familial connections, possibly pointing to narratives or poems dealing with personal relationships, identity, and the human condition reflected through physical and relational descriptions. Conclusion: Emphasizing interpersonal relationships and the intimate aspects of human experiences, this topic could explore the profound connections between individuals, perhaps within familial or close-knit community settings.

Note: used chatgpt for drawing the conclusion and it required considerable prompt engineering


In [ ]:
lda_model, bow, diction = choherence_perplexity_calculator(TSE_list)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[['title', 'macavity', 'the', 'mystery', 'cat', 'poem', 'macavity', 'mystery', 'cat', 'he', 'is', 'called', 'the', 'hidden', 'paw', 'for', 'he', 'is', 'the', 'master', 'criminal', 'who', 'can', 'defy', 'the', 'law', 'he', 'is', 'the', 'bafflement', 'of', 'scotland', 'yard', 'the', 'flying', 'squad', 'despair', 'for', 'when', 'they', 'reach', 'the', 'scene', 'of', 'crime', 'macavity', 'not', 'there', 'macavity', 'macavity', 'there', 'is', 'no', 'one', 'like', 'macavity', 'he', 'is', 'broken', 'every', 'human', 'law', 'he', 'breaks', 'the', 'law', 'of', 'gravity', 'his', 'powers', 'of', 'levitation', 'would', 'make', 'fakir', 'stare', 'and', 'when', 'you', 'reach', 'the', 'scene', 'of', 'crime', 'macavity', 'not', 'there', 'you', 'may', 'seek', 'him', 'in', 'the', 'basement', 'you', 'may', 'look', 'up', 'in', 'the', 'air', 'but', 'tell', 'you', 'once', 'and', 'once', 'again', 'macavity', 'not', 'there', 'macavity', 'ginger', 'cat', 'he', 'is', 'very', 'tall', 'and', 'thin', 'you', 'would

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


[(0, '0.070*"eye" + 0.040*"tear" + 0.021*"death" + 0.021*"affliction" + 0.021*"kingdom" + 0.011*"door" + 0.011*"title" + 0.011*"division" + 0.011*"vision" + 0.011*"decision"'), (1, '0.046*"time" + 0.014*"hair" + 0.014*"night" + 0.014*"window" + 0.014*"death" + 0.014*"evening" + 0.014*"street" + 0.011*"arm" + 0.011*"birth" + 0.008*"moment"'), (2, '0.001*"day" + 0.001*"work" + 0.001*"time" + 0.001*"water" + 0.001*"jennyanydot" + 0.001*"bed" + 0.001*"mind" + 0.001*"man" + 0.001*"window" + 0.001*"name"'), (3, '0.002*"time" + 0.002*"water" + 0.001*"rock" + 0.001*"mountain" + 0.001*"night" + 0.001*"man" + 0.001*"hand" + 0.001*"room" + 0.001*"friend" + 0.001*"eye"'), (4, '0.035*"friend" + 0.020*"afternoon" + 0.016*"hand" + 0.016*"life" + 0.012*"room" + 0.012*"smile" + 0.012*"tea" + 0.012*"end" + 0.012*"self" + 0.012*"violin"'), (5, '0.021*"water" + 0.014*"rock" + 0.012*"time" + 0.011*"man" + 0.010*"mountain" + 0.009*"night" + 0.008*"eye" + 0.007*"day" + 0.007*"window" + 0.007*"sound"'), (6, '

In [ ]:
gensimvis.prepare(lda_model, bow, diction)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
5      0.018879  0.069586       1        1  56.406136
1      0.181414  0.023231       2        1  21.001678
4     -0.028909 -0.198140       3        1  12.890935
7     -0.137599  0.080657       4        1   7.672576
0     -0.006671  0.018427       5        1   1.750814
3     -0.004805  0.001118       6        1   0.055572
6     -0.004774  0.000222       7        1   0.055572
2     -0.005402  0.001748       8        1   0.055572
9     -0.006653  0.001496       9        1   0.055572
8     -0.005480  0.001655      10        1   0.055572, topic_info=         Term       Freq      Total Category  logprob  loglift
20        eye  10.000000  10.000000  Default  30.0000  30.0000
36   macavity   9.000000   9.000000  Default  29.0000  29.0000
70       time  24.000000  24.000000  Default  28.0000  28.0000
166     death   8.000000   8.000000  Default  27.0000  27.0000
214    friend   9.000000   9.000000  Default  26.0000  26.0000
..        ...        ...        ...      ...      ...      ...
237      hand   0.001054   9.286954  Topic10  -6.6170  -1.5886
140      city   0.001053   5.681365  Topic10  -6.6175  -1.0977
397     smoke   0.001052   5.022921  Topic10  -6.6191  -0.9761
73        way   0.001051   5.117727  Topic10  -6.6200  -0.9957
86        arm   0.001050   4.411814  Topic10  -6.6204  -0.8476

[732 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
671       3  0.968202     achille
0         4  1.091742   admiralty
672       3  0.968705      admire
673       3  0.968366   advantage
632       5  1.220742  affliction
...     ...       ...         ...
539       1  0.778028        work
539       2  0.259343        work
76        4  1.091598        yard
495       1  0.910440        year
495       2  0.151740        year

[404 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[6, 2, 5, 8, 1, 4, 7, 3, 10, 9])

In [ ]:
lda_model.top_topics(bow)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[([(0.08642427, 'macavity'),
   (0.03273191, 'cat'),
   (0.021999396, 'crime'),
   (0.016681993, 'one'),
   (0.01664299, 'law'),
   (0.011330192, 'time'),
   (0.011277699, 'scene'),
   (0.01127713, 'head'),
   (0.011264735, 'deed'),
   (0.011261519, 'mystery'),
   (0.0059663267, 'eye'),
   (0.005937365, 'air'),
   (0.0059338734, 'glass'),
   (0.005921124, 'stair'),
   (0.0059208605, 'card'),
   (0.005917434, 'jewel'),
   (0.0059156804, 'side'),
   (0.0059156464, 'loss'),
   (0.0059133396, 'stare'),
   (0.005912023, 'paper')],
  -0.3497691121665341),
 ([(0.020595191, 'water'),
   (0.013764951, 'rock'),
   (0.011502108, 'time'),
   (0.011489901, 'man'),
   (0.010349694, 'mountain'),
   (0.009213558, 'night'),
   (0.008072751, 'eye'),
   (0.006952354, 'day'),
   (0.0069458806, 'window'),
   (0.0069426396, 'sound'),
   (0.0069410685, 'hour'),
   (0.006939892, 'foot'),
   (0.006936578, 'year'),
   (0.005808255, 'air'),
   (0.0058055976, 'bed'),
   (0.005805069, 'hand'),
   (0.0058034873, 'd

### Take Away

Topic with 'macavity', 'cat', 'crime', etc.:
This cluster clearly revolves around the character "Macavity" from T.S. Eliot's poem "Macavity: The Mystery Cat". The presence of terms like "crime," "mystery," "law," and "scene" reinforces the portrayal of Macavity as a criminal mastermind elusive to law enforcement. Conclusion: This topic captures the essence of the playful yet intriguing depiction of a criminal cat, highlighting themes of cunning, elusiveness, and the whimsical subversion of typical criminal stories.

Topic with 'water', 'rock', 'time', etc.:
Words like "water," "rock," "mountain," and "night" suggest a natural setting, potentially evoking themes of the natural world's enduring and unyielding characteristics. Conclusion: This topic likely explores nature's vastness and timelessness, possibly reflecting on human experiences or emotions in contrast with nature's permanence and immensity.

Topic with minimal differences in terms like 'time', 'night', 'water', etc.:
This set appears to emphasize the continual themes of time and nature but with a deeper focus on more abstract or introspective elements like "death," "woman," and "evening." Conclusion: This topic delves into the cyclical nature of life, the inevitable passage of time, and the introspective moments that define human existence, often set against a backdrop of natural imagery.

Topic with 'friend', 'afternoon', 'hand', etc.:
Centering around social interactions and personal relationships, terms like "friend," "afternoon," "tea," and "smile" suggest a setting of leisure and intimacy. Conclusion: This topic might represent scenes from social gatherings or personal reflections within friendships, offering a warm, inviting atmosphere conducive to sharing and personal reflection.

Topic with 'time', 'hair', 'night', etc.:
With a heavy focus on "time," "night," and "death," and integrating physical elements like "hair" and "street," this cluster seems to weave together the personal and the existential. Conclusion: Reflecting on the passage of time and its impact on personal identity and mortality, this topic may explore how personal moments are framed within the broader context of life's transience.

Topic with 'water', 'friend', 'death', etc.:
Incorporating both natural elements and human relationships, this cluster hints at narratives or themes where personal experiences intersect with natural settings, perhaps exploring life's transitional or transformative moments. Conclusion: This topic likely explores themes of journey and transformation, possibly metaphorically using elements like "water" and "rock" to symbolize life's challenges and changes.

Topic with diverse terms ('macavity', 'time', 'cat', etc.):
This cluster is broad, including "Macavity," "time," "crime," and "night," suggesting a blend of narrative storytelling with thematic exploration of time and moral ambiguities. Conclusion: This might reflect a comprehensive narrative style that blends character-focused storytelling with broader philosophical questions about morality and time.

Topic with 'eye', 'tear', 'death', etc.:
Focusing on highly emotive terms like "eye," "tear," and "death," this cluster intensely explores themes of sorrow, loss, and reflection. Conclusion: Emphasizing the emotional responses to life's hardships, this topic delves into the depths of human grief, resilience, and the emotional spectrum associated with loss and remembrance.

In [ ]:
def ner(text):
  return spacy.load("en_core_web_sm")(text)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
original = ner(RP_input['0']['Poem'])

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
displacy.render(original, style="ent", jupyter=True)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
